In [1]:
!pip install -q docling docling-core

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 4.1 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.1/166.1 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.3/142.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 85.3 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.7/73.7 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 82.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 28

In [2]:
from docling.document_converter import DocumentConverter, PdfFormatOption
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import PdfPipelineOptions, TableFormerMode
import time

2025-05-11 10:52:48.626834: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746960768.822100      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746960768.877112      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## 1. Khởi tạo Docling converter

In [3]:
# Define pipeline options for PDF processing
pipeline_options = PdfPipelineOptions(
    do_table_structure=True,  # Enable table structure detection
    do_ocr=False,  # Enable OCR
    # full page ocr and language selection
    #ocr_options=EasyOcrOptions(force_full_page_ocr=True, lang=["en"]),  # Use EasyOCR for OCR
    # ocr_options=TesseractOcrOptions(force_full_page_ocr=True, lang=["eng"]),  # Uncomment to use Tesseract for OCR
    #ocr_options = OcrMacOptions(force_full_page_ocr=True, lang=['en-US']),
    table_structure_options=dict(
        do_cell_matching=False,  # Use text cells predicted from table structure model
        mode=TableFormerMode.ACCURATE # Use more accurate Tableformer model
    ),
    generate_page_images=False,  # Enable page image generation
    generate_picture_images=False,  # Enable picture image generation
    # images_scale=IMAGE_RESOLUTION_SCALE,  # Set image resolution scale (scale=1 corresponds to a standard 72 DPI image)
)


format_options = {
    InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
}
converter = DocumentConverter(format_options=format_options)

def convert_pdf_to_docling(sources: list[str]):
    start_time = time.time()
    conversions = {
        source: converter.convert(source).document for source in sources
    }
    end_time = time.time()
    print(f"Time taken: {end_time - start_time:.4f}")
    return conversions

In [5]:
# sources = [
#     "CA Warn Report.pdf",
#     # "/content/drive/MyDrive/Docling/national-capitals.pdf",
#     # "/content/drive/MyDrive/Docling/c935e2902adf7040a6ffe0db0f7c11e6.pdf",
# ]

# conversions = convert_pdf_to_docling(sources)

## 2. Các hàm hữu ích

In [6]:
import csv
import pandas as pd
from io import StringIO
import numpy as np


def preprocess_df(df):
    # Check if columns are numeric sequence
    if all(str(col).isdigit() for col in df.columns):
        # Get first row as new columns
        new_cols = df.iloc[0].values
        # Drop first row and reset columns
        df = df.iloc[1:].reset_index(drop=True)
        df.columns = new_cols
        
    # Check if first row is numeric sequence
    elif df.shape[0] > 0:
        first_row = df.iloc[0]
        try:
            # Convert to numeric and check if it's a sequence
            nums = pd.to_numeric(first_row)
            if all(nums[i] == i for i in range(len(nums))):
                # Drop first row if it's a numeric sequence
                df = df.iloc[1:].reset_index(drop=True)
        except:
            pass
            
    return df



def _perform_csv_sniffing(csv_sample_string: str) -> bool:
    """Kiểm tra sự tồn tại của header trong chuỗi CSV mẫu bằng csv.Sniffer."""
    if not csv_sample_string.strip():
        return False
    sniffer = csv.Sniffer()
    try:
        return sniffer.has_header(csv_sample_string)
    except csv.Error: # Lỗi định dạng CSV không xác định được
        return False

def has_header(data_source: any) -> bool:
    """
    Kiểm tra DataFrame (hoặc file CSV) có header không.
    Trả về: True nếu có header, False nếu không.
    """
    MAX_SAMPLE_ROWS_FOR_DF = 15

    if isinstance(data_source, pd.DataFrame):
        df = data_source
        if df.empty:
            return False

        columns = df.columns.tolist()
        if not columns: # Không có cột, không có header
            return False

        # Loại trừ cột là số nguyên tuần tự (0,1,2...) làm header
        is_sequential_integers = True
        for i, col_name_obj in enumerate(columns):
            try:
                if int(str(col_name_obj)) != i:
                    is_sequential_integers = False
                    break
            except ValueError:
                is_sequential_integers = False
                break
        if is_sequential_integers:
            return False

        # Loại trừ cột 'Unnamed: X' làm header
        for col in columns:
            if str(col).startswith('Unnamed'):
                return False
        
        # Dùng CSV sniffing trên mẫu DataFrame
        sample_df = df.head(MAX_SAMPLE_ROWS_FOR_DF)
        
        # (Nếu DataFrame chỉ có tên cột đã qua kiểm tra, vẫn coi là có header; logic tiếp tục)

        csv_string_buffer = StringIO()
        # Chuyển mẫu DataFrame (kèm header) sang CSV để sniff
        sample_df.to_csv(csv_string_buffer, header=True, index=False)
        csv_sample_from_df = csv_string_buffer.getvalue()
        
        return _perform_csv_sniffing(csv_sample_from_df)

    elif isinstance(data_source, str): # Xử lý trường hợp data_source là đường dẫn file CSV
        MAX_FILE_LINES_FOR_SAMPLE = 15 # Giới hạn số dòng đọc cho mẫu file
        try:
            with open(data_source, 'r', newline='', encoding='utf-8-sig') as csv_file:
                sample_lines_list = [line for i, line in enumerate(csv_file) if i < MAX_FILE_LINES_FOR_SAMPLE]
                if not sample_lines_list:
                    return False
                return _perform_csv_sniffing("".join(sample_lines_list))
        except FileNotFoundError:
            return False 
        except Exception: # Lỗi đọc file khác
            return False
    else: # Kiểu dữ liệu không được hỗ trợ
        return False

def solve_non_header_table(df: pd.DataFrame, headers: list) -> pd.DataFrame:
    """
    Chuyển đổi DataFrame không có header: tên cột cũ thành dòng dữ liệu đầu tiên, áp dụng `headers` mới.
    Cột 'Unnamed' cũ thành NaN trong dòng dữ liệu mới.

    Args:
        df (pd.DataFrame): DataFrame đầu vào (không có header).
        headers (list): Danh sách header mới.
    Returns:
        pd.DataFrame: DataFrame đã điều chỉnh.
    """
    df_copy = df.copy() # Làm việc trên bản sao

    first_row_values = []
    for col_original_name in df_copy.columns:
        if str(col_original_name).startswith('Unnamed'):
            first_row_values.append(np.nan)
        else:
            first_row_values.append(col_original_name) # Tên cột cũ thành dữ liệu
            
    # Đảm bảo first_row_values khớp độ dài headers.
    # Logic gọi hàm main_concatenation_logic thường đảm bảo len(df.columns) == len(headers).
    if len(first_row_values) != len(headers):
        if len(first_row_values) > len(headers): # Cắt bớt nếu thừa
            first_row_values = first_row_values[:len(headers)]
        else: # Thêm NaN nếu thiếu
            first_row_values.extend([np.nan] * (len(headers) - len(first_row_values)))

    temp_firstrow_df = pd.DataFrame([first_row_values], columns=headers) # Dòng dữ liệu mới từ tên cột cũ
    df_copy.columns = headers # Áp dụng header mới cho dữ liệu gốc
    
    result_df = pd.concat([temp_firstrow_df, df_copy], ignore_index=True) # Ghép lại
    return result_df.reset_index(drop=True)

def get_input_df(df, n_rows, sep="="):
        columns = df.columns.tolist()
        # Check if any columns start with "Unnamed" or are numeric indices
        unnamed_cols = [col for col in columns if str(col).startswith("Unnamed")]
        # check columns is all digits
        numeric_cols = all(
            isinstance(col, int) or (isinstance(col, str) and col.isdigit())
            for col in columns
        )
        # check if NAN exists in columns
        nan_cols = any(pd.isna(col) for col in df.columns)
        header = not (
            bool(unnamed_cols) or numeric_cols or nan_cols or not has_header(df)
        )  # Don't show header if unnamed or numeric columns or has NaN columns

        if len(df) < n_rows:
            if header:
                return f"Headers: {','.join(columns)}\n" + df.to_csv(
                    index=False, header=False, sep=sep
                )
            return df.to_csv(index=False, header=False, sep=sep)
        else:
            # Calculate number of rows for each section
            n_edge = n_rows // 3  # Number of rows to take from start and end
            n_random = n_rows - (2 * n_edge)  # Remaining rows will be random

            # Get first and last n_edge rows
            first_rows = df.head(n_edge)
            last_rows = df.tail(n_edge)
            # Get random rows from middle section
            middle_section = df.iloc[n_edge:-n_edge]
            random_rows = (
                middle_section.sample(n=n_random)
                if len(middle_section) >= n_random
                else middle_section
            )

            concated_df = pd.concat([first_rows, last_rows, random_rows])
            if header:
                return f"Headers: {','.join(columns)}\n" + concated_df.to_csv(
                    index=False, header=False, sep=sep
                )
            return concated_df.to_csv(index=False, header=False, sep=sep)

def get_column_types(df):
    types = []
    if df.columns.empty:
        return types

    for col_name in df.columns: 
        current_column_object = df[col_name]

        is_truly_all_na = False
        effective_dtype = None

        if isinstance(current_column_object, pd.DataFrame):
            is_truly_all_na = pd.isna(current_column_object).all().all()
            if not is_truly_all_na:
                effective_dtype = current_column_object.iloc[:, 0].dtype
        elif isinstance(current_column_object, pd.Series):
            is_truly_all_na = pd.isna(current_column_object).all()
            if not is_truly_all_na:
                effective_dtype = current_column_object.dtype
        else:
            types.append("error_unexpected_type")
            continue

        if is_truly_all_na:
            types.append("unknown")
        else:
            if effective_dtype is not None:
                types.append(
                    "numeric" if pd.api.types.is_numeric_dtype(effective_dtype) else str(effective_dtype)
                )
            else:
                types.append("unknown_error_in_logic")
    return types

## 3. Xử lý

Duyệt qua Docling và lấy các tables dưới dạng `pd.DataFrame` và tạo ra `input_prompt` cho LLM

In [7]:
from docling_core.types.doc.labels import DocItemLabel
from docling_core.transforms.chunker.hybrid_chunker import HybridChunker
from docling_core.types.doc.document import TableItem
from langchain_core.documents import Document
import pandas as pd

def get_tables_from_docling(docling_document) -> tuple[list[pd.DataFrame], str]:
    doc_id = 0
    tables: list[pd.DataFrame] = []
    full_prompt = ""
    for table in docling_document.tables:
        if table.label in [DocItemLabel.TABLE]:
            # ref = table.get_ref().cref
            # text = table.export_to_markdown(docling_document)
            df = preprocess_df(table.export_to_dataframe())
            df_string = get_input_df(df, n_rows=10)
            column_types = get_column_types(df=df)
            n_columns = len(df.columns.to_list())
            full_prompt += f"Table {doc_id}\n{df_string}\nNumber of columns: {n_columns}\nColumn types: {column_types}\n\n"
            tables.append(df)
            doc_id += 1

    print(f"{len(tables)} tables extracted")
    return tables, full_prompt

In [8]:
# tables, full_prompt = get_tables_from_docling(conversions[sources[0]])

### Hàm call LLM API

In [9]:
# To run this code you need to install the following dependencies:
# pip install google-genai

import base64
import os
import json
from google import genai
from google.genai import types
# from google.colab import userdata
# GEMINI_API_KEY = userdata.get('GOOGLE_API_KEY')
# from dotenv import load_dotenv
# load_dotenv()

def generate(prompt):
    model = "gemini-2.0-flash-lite"
    client = genai.Client(
        api_key=GEMINI_API_KEY,
    )
    system_prompt = """You are an AI assistant specialized in analyzing and grouping data tables for vertical concatenation. I will provide you with information for multiple tables. Each table will include:

Table index (e.g., \"Table 0\").
Explicit headers (e.g., \"Headers: ColA,ColB,ColC\") if this table introduces a new header structure. For subsequent tables with the same structure and headers, the \"Headers:\" line can be omitted.
A few lines of sample data. Each data line contains values separated by the \"=\" character.
The number of columns.
A list of data types for the columns.
The tables will be presented sequentially.

Your task is to identify groups of tables that can be vertically concatenated. Tables are considered concatenable if they share the following characteristics:

The same number of columns.
Compatible data types for corresponding columns.
Identical or very similar headers. If headers are not explicitly provided for a table, assume it shares the headers of the immediately preceding table (or the closest table with explicit headers) if their number of columns and data types match.
Below is the table data:"""
    content = system_prompt + prompt
    contents = [
        types.Content(
            role="user",
            parts=[
                types.Part.from_text(text=prompt),
            ],
        ),
    ]
    generate_content_config = types.GenerateContentConfig(
        temperature=0,
        response_mime_type="application/json",
        response_schema=genai.types.Schema(
            type = genai.types.Type.OBJECT,
            required = ["concatable_tables"],
            properties = {
                "concatable_tables": genai.types.Schema(
                    type = genai.types.Type.ARRAY,
                    items = genai.types.Schema(
                        type = genai.types.Type.OBJECT,
                        required = ["table_index"],
                        properties = {
                            "table_index": genai.types.Schema(
                                type = genai.types.Type.ARRAY,
                                items = genai.types.Schema(
                                    type = genai.types.Type.NUMBER,
                                ),
                            ),
                        },
                    ),
                ),
            },
        ),
        system_instruction=[
            types.Part.from_text(text=system_prompt),
        ],
    )
    input_tokens = client.models.count_tokens(model=model, contents=[content])
    response = client.models.generate_content(
        model=model,
        contents=contents,
        config=generate_content_config,
    )
    output_tokens = client.models.count_tokens(model=model, contents=response.text)
    print(f"Input tokens: {input_tokens}")
    print(f"Output tokens: {output_tokens}")

    json_response = json.loads(response.text)
    print(json_response)
    return json_response


/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:623: UserWarning: <built-in function any> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


In [10]:
# json_instructions = generate(full_prompt)
# print(json_instructions)

Ghép nối các nhóm tables lại với nhau

In [20]:
def main_concatenation_logic(tables: list[pd.DataFrame], concat_json: dict) -> list[pd.DataFrame]:
    """
    Ghép các DataFrame theo nhóm chỉ định trong `concat_json`.
    Sử dụng `has_header` và `solve_non_header_table` để chuẩn hóa header.
    Các bảng không được đề cập trong `concat_json` sẽ được coi là độc lập và thêm vào kết quả.

    Args:
        tables (list[pd.DataFrame]): Danh sách tất cả DataFrame.
        concat_json (dict): Hướng dẫn ghép bảng, ví dụ:
                            {'concatable_tables': [{'table_index': [0, 1]}, ...]}
    Returns:
        list[pd.DataFrame]: Danh sách các DataFrame đã ghép và các DataFrame độc lập.
    """
    concatenated_results = [] # Lưu kết quả ghép
    processed_indices_from_json = set() # Theo dõi các chỉ mục đã được xử lý từ concat_json
    
    # --- Kiểm tra đầu vào cơ bản ---
    if not isinstance(tables, list) or not all(isinstance(t, pd.DataFrame) for t in tables):
        print("Lỗi: 'tables' phải là list[pd.DataFrame].") # Cân nhắc dùng raise ValueError
        return [] 
    if not isinstance(concat_json, dict) or 'concatable_tables' not in concat_json:
        # Nếu concat_json không hợp lệ hoặc không có 'concatable_tables',
        # tất cả các bảng sẽ được coi là độc lập.
        # Hoặc bạn có thể trả về lỗi/mảng rỗng tùy theo yêu cầu.
        # Ở đây, tôi sẽ tiếp tục để xử lý tất cả như bảng độc lập nếu key bị thiếu.
        # Nếu chỉ là concat_json['concatable_tables'] rỗng thì vòng lặp sẽ không chạy, cũng ổn.
        if not isinstance(concat_json, dict) or 'concatable_tables' not in concat_json.get('concatable_tables', [{}]): # Kiểm tra chặt chẽ hơn một chút
            print("Cảnh báo: 'concat_json' không hợp lệ hoặc không có 'concatable_tables'. Tất cả bảng sẽ được coi là độc lập nếu có.")
            # Cho phép tiếp tục để xử lý các bảng độc lập nếu `tables` có dữ liệu
            pass


    # --- Xử lý các nhóm được định nghĩa trong concat_json ---
    for group_instruction in concat_json.get('concatable_tables', []): # Xử lý từng nhóm ghép
        table_indices_from_json_group = group_instruction.get('table_index', [])
        if not table_indices_from_json_group: # Bỏ qua nhóm rỗng trong JSON
            continue

        current_group_dfs_to_process = []
        valid_indices_in_this_json_group = [] 

        # Tập hợp các DataFrame hợp lệ cho nhóm hiện tại từ JSON
        for index in table_indices_from_json_group:
            if 0 <= index < len(tables):
                current_group_dfs_to_process.append(tables[index])
                valid_indices_in_this_json_group.append(index)
            # else: Bỏ qua chỉ mục ngoài giới hạn từ JSON (không thông báo để code ngắn gọn)

        if not current_group_dfs_to_process:
            continue
        
        # Đánh dấu các chỉ mục này là đã được xử lý thông qua JSON
        for idx in valid_indices_in_this_json_group:
            processed_indices_from_json.add(idx)
            
        # Xác định header chuẩn cho nhóm hiện tại (nếu có)
        authoritative_headers = None
        for df_check in current_group_dfs_to_process: # df_check là DataFrame gốc trong nhóm
            if has_header(df_check): 
                authoritative_headers = df_check.columns.tolist()
                break
        
        prepared_dfs_for_concat = [] # Các DataFrame đã xử lý, sẵn sàng để ghép cho nhóm này
        for original_df_in_group in current_group_dfs_to_process:
            df_to_process = original_df_in_group.copy() # Luôn xử lý bản sao

            if authoritative_headers is not None: # Trường hợp nhóm có header chuẩn
                if has_header(df_to_process): # Bảng hiện tại có header
                    if len(df_to_process.columns) == len(authoritative_headers):
                        df_to_process.columns = authoritative_headers # Cùng số cột: đổi tên theo header chuẩn
                    # else: Khác số cột, giữ header gốc, pd.concat sẽ tự xử lý
                else: # Bảng hiện tại không có header
                    if len(df_to_process.columns) == len(authoritative_headers):
                        df_to_process = solve_non_header_table(df_to_process, authoritative_headers) # Cùng số cột: dùng solve_non_header_table
                    # else: Khác số cột, không xử lý, pd.concat sẽ tự xử lý
            # else: Trường hợp nhóm không có header chuẩn (tất cả bảng trong nhóm đều headerless) -> ghép nguyên trạng
            
            prepared_dfs_for_concat.append(df_to_process)

        if prepared_dfs_for_concat: # Thực hiện ghép nếu có bảng đã xử lý trong nhóm
            final_df_group = pd.concat(prepared_dfs_for_concat, ignore_index=True)
            concatenated_results.append(final_df_group)
            
    # --- Xử lý các bảng độc lập không được đề cập trong concat_json ---
    all_available_indices = set(range(len(tables)))
    # Lấy các chỉ mục không có trong processed_indices_from_json
    independent_table_indices = sorted(list(all_available_indices - processed_indices_from_json))

    for idx in independent_table_indices:
        # Các bảng này không thuộc bất kỳ nhóm nào do API chỉ định.
        # Chúng được thêm vào kết quả dưới dạng các mục riêng lẻ.
        # Trạng thái (header, v.v.) của chúng được lấy nguyên trạng từ danh sách `tables` đầu vào,
        # giả định rằng mọi tiền xử lý cần thiết cho từng bảng đã được thực hiện trước khi gọi hàm này.
        concatenated_results.append(tables[idx].copy()) 
            
    return concatenated_results

### Kết quả

In [12]:
# concatenated_dataframes = main_concatenation_logic(tables, json_instructions)
# print(f"Có tổng cộng {len(concatenated_dataframes)} bảng!")

## 4. Full pipeline

In [21]:
from IPython.display import Markdown, display

def get_tables_from_sources(sources: list[str], output_path: str = None) -> dict[str, list[pd.DataFrame]]:
    processed_tables = {}
    conversions = convert_pdf_to_docling(sources)
    
    for source in sources:
        pdf_name = source.split("/")[-1][:-4]
        print(f"Processing {pdf_name}")
        try:
            tables, full_prompt = get_tables_from_docling(conversions[source])
            json_instructions = generate(full_prompt)
            concatenated_dataframes = main_concatenation_logic(tables, json_instructions)
            processed_tables[source] = concatenated_dataframes
            if output_path:
                for i, df in enumerate(concatenated_dataframes):
                    df.to_csv(f"{pdf_name}_{i}.csv", index=False)
                    print(f"\n--- Concatenated DataFrame Group {i+1} ---")
                    display(Markdown(df.to_markdown()))
                    print(f"Columns: {df.columns.tolist()}")
                    print(f"Shape: {df.shape}")
            print(f"Processed {pdf_name}")
            print("="*50)
        except Exception as e:
            print(f"Error processing {source}: {e}")
    return processed_tables
    

In [22]:
### SETTING INPUT AND OUTPUT PATH
import os

input_path = "/kaggle/input/tables-pdf"

sources = [os.path.join(input_path, f) for f in os.listdir(input_path) if f.endswith(".pdf")]
print("Source:")
print(sources)

pdf_names = [pdf_path.split("/")[-1] for pdf_path in sources]
print(pdf_names)
output_path = "/kaggle/working/"
print("Output path")
print(output_path)

Source:
['/kaggle/input/tables-pdf/c935e2902adf7040a6ffe0db0f7c11e6.pdf', '/kaggle/input/tables-pdf/national-capitals.pdf', '/kaggle/input/tables-pdf/CA Warn Report.pdf', '/kaggle/input/tables-pdf/659dfd3e5c22a64d7597cb50c6fc455e.pdf', '/kaggle/input/tables-pdf/0e6af4a364ff3fe835bd4e332a2fb9cf.pdf', '/kaggle/input/tables-pdf/f8a04d6beeed137eeb7840a5c459842d.pdf', '/kaggle/input/tables-pdf/78f480a26ad6a8c7c064e9d9fd5b2c4f.pdf']
['c935e2902adf7040a6ffe0db0f7c11e6.pdf', 'national-capitals.pdf', 'CA Warn Report.pdf', '659dfd3e5c22a64d7597cb50c6fc455e.pdf', '0e6af4a364ff3fe835bd4e332a2fb9cf.pdf', 'f8a04d6beeed137eeb7840a5c459842d.pdf', '78f480a26ad6a8c7c064e9d9fd5b2c4f.pdf']
Output path
/kaggle/working/


In [23]:
processed_tables = get_tables_from_sources(sources, output_path=output_path)

Time taken: 170.8810
Processing c935e2902adf7040a6ffe0db0f7c11e6
10 tables extracted
Input tokens: total_tokens=4131 cached_content_token_count=None
Output tokens: total_tokens=64 cached_content_token_count=None
{'concatable_tables': [{'table_index': [0, 1, 2, 3, 4, 5, 6, 7]}]}

--- Concatenated DataFrame Group 1 ---


|    | Air Jordan IV      |   1989 | In December 1988, Nike released the Air Jordan IV to the public. Designed by Tinker Hatfield, it was the first Air Jordan released on the global market. It had four colorways: White/Black, Black/Cement Grey, White/Fire Red-Black, and Off White/Military Blue. Nike featured director and actor Spike Lee in ads for the shoe. [27]  Lee had featured the shoe in his movie  Do The Right Thing . [25] Michael Jordan wore the Air Jordan IV when he made "The Shot", a series winner in Game 5 of the 1989 NBA First Round between the Chicago Bulls and the Cleveland Cavaliers. In 2012 a Cavalier colorway dubbed the "Cavs" was released to honor "The Shot".   |
|---:|:-------------------|-------:|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
|  0 | Air Jordan V       |   1990 | The Air Jordan V was released in February 1990 and designed by Hatfield. Inspired by a WWII Mustang fighter, [25] features include a reflective tongue with a protruding design, translucent rubber soles and lace locks. [28] The Air Jordan V saw a lot of use in popular sitcom  The Fresh Prince of Bel-Air . During many episodes Will Smith wore the Metallic Silver, Grape, and Fire Red colorways. To pay tribute to his character, Jordan released the Air Jordan 5 Bel Air in 2013 and 2020. [29][30]                                                                                                                                                          |
|  1 | Air Jordan VI      |   1991 | Designed after a German sports car, Michael Jordan wore the VI for his first Bulls championship. [25]  The shoes were worn by the protagonist of the manga  Slam Dunk , Hanamichi Sakuragi. In 2014, Nike released special versions of the VI featuring artwork from the series. [31]  Various other models in the Air Jordan line are featured in the series, including the original Air Jordan, the V, and the XII. [32]                                                                                                                                                                                                                                               |
|  2 | Air Jordan VII     |   1992 | The Air Jordan VII introduced "huarache" technology which allowed the shoes to better conform to the user's foot. A few things were no longer featured on the new model, such as the visible air sole, the Nike Air logo, and the translucent soles. This was the first Air Jordan in the line that did not have any distinctive "Nike Air" branding on the outer portions of the shoe, only on the in-soles. When Jordan went to compete at the 1992 Summer Olympics to play for the US Men's Basketball Team (also known as the "Dream Team"), Nike released a special                                                                                                 |
|  3 | Air Jordan VIII    |   1993 | The Air Jordan VIII was released to coincide with the 1992- 93 NBA season. The eighth model of the Air Jordan contains a full-length air sole, polyurethane midsole, polycarbonate shank plate, and two crossover straps. The VIIIs were known for a successful ad campaign in which Bugs Bunny appeared alongside Michael Jordan to market the shoes. [25]  The shoe was re-released in 2003, 2007, 2008, 2013, and 2015-2017.                                                                                                                                                                                                                                          |
|  4 | Air Jordan IX      |   1993 | Originally released in November 1993, the Air Jordan IX was the first model released after Michael Jordan's retirement. Jordan never played an NBA season wearing these shoes. This model was inspired by baseball cleats that Jordan wore when playing minor-league baseball. [25]  The shoe was re- released in 2002, 2008, 2010, 2012, and 2014-2018. Like the VII and VIII models, the Air Jordan IX featured an inner sock sleeve and nubuck accents. The sole featured different symbols and languages of different countries. The Air Jordan IX is depicted to be worn by the Statue of Michael Jordan inside the United Center in Chicago. [34]                  |
|  5 | Air Jordan X       |   1994 | This was released in different colors representing U.S. cities. It was the first Air Jordan to feature a lightweight Phylon midsole. The shoe also featured all of Michael Jordan's accomplishments up to his first retirement on the outsole. [25] The shoe was re-released in 2005, 2008, 2012-2016, and 2018.                                                                                                                                                                                                                                                                                                                                                         |
|  6 | Air Jordan XI      |   1995 | This model was designed by Tinker Hatfield. When the shoe launched, Michael Jordan (retired from basketball by then) was with the Birmingham Barons in baseball's minor baseball leagues. Hatfield designed the sneaker waiting for Jordan to come back and hoping he would play in them. [35] The ballistic mesh upper of the sneaker was meant to make the Air Jordan XI lighter and more durable. Further changes                                                                                                                                                                                                                                                     |
|  7 |                    |   1997 | This model had a carbon fiber plate, designed by Hatfield. The black panther was the inspiration for the Air Jordan XIII, with the sole resembling the pads on a panther's paw. A hologram on the back of the shoe imitates a panther's eyes in the dark. [25]  They were re-released in 2005, which coincided with the release of the Air Jordan 8s shoe. In the movie  He Got Game , director Spike Lee had access to the Air Jordan XIII months before it was available to the public or even worn by Jordan himself and featured it in scenes. [37]                                                                                                                  |
|  8 | Air Jordan XIV     |   1998 | Inspired by the Ferrari 550 M which Michael Jordan owned, the Air Jordan XIV was originally released in October 1998. [25]  It was re-released in 2005, 2006, 2008, 2011, 2012 and 2014-2018. The Air Jordan XIV co-styled by Hatfield and Mark Smith was made race ready and equipped with the Jordan Jumpman insignia on a Ferrari shaped badge. The color scheme of predominant black accentuated with red was nicknamed "The Last Shot" because Michael Jordan wore them as he hit the game winning shot over Bryon Russell, of the Utah Jazz,                                                                                                                       |
|  9 | Air Jordan XV      |   1999 | This was the first shoe after Jordan's retirement. The design was inspired by the North American X-15, which was developed by NASA during the 1950s. The sides of the XV were made from woven kevlar fiber. [25]  The Jordan XVs were Hatfield's least favorite in the series. [38]                                                                                                                                                                                                                                                                                                                                                                                      |
| 10 | Air Jordan XVI     |   2001 | The shoe came with spats, and the design was inspired by performance cars and architecture. The ad campaign featured Mos Def. [25]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 11 | Air Jordan XVII    |   2002 | This pair of Jordans came with a multimedia CD-ROM containing the Air Jordan XVII song. The retail price of the shoe was US$200. [25]  The defining functional design element of the Air Jordan XVII model, which was later replicated on the Air Jordan XXIII model, was the reinforced mid-sole which provided a sturdy and stable chassis for the shoe. They were made in four mid top colors and three low-top colors. Michael Jordan wore the XVII while playing for the Washington Wizards, after his second retirement return. [41] The shoe was re-released in 2008 and 2016.                                                                                    |
| 12 | Air Jordan XVIII   |   2003 | The Air Jordan XVIII shoe was released during Michael Jordan's last season, in which he played for the Washington Wizards. The shoe was designed by Air Jordan Senior Footwear Designer, Tate Kuerbis, who had been part of the Jordan footwear design team since 1999 and with Nike since 1995. The inspiration for the design came from the carbon fiber- based monocoque of F1 race cars, [25]  race car driving shoes (rubber heel wrap) and Fine Italian dress shoes (bold stitching on the soles). It was re-released in 2008.                                                                                                                                     |
| 13 |                    |   2007 | The XX2 was inspired by the F-22 Raptor. [25]  The promo commercial was directed by Mark Romanek. [44]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   |
| 14 | Air Jordan XX3     |   2008 | The Air Jordan XX3 was designed by Tinker Hatfield. It was the first basketball shoe in the "Nike Considered" category, for using materials from not more than 200 miles (320 km) from a Nike Factory. It features a hand-stitched exterior, full- length bootie, carbon fiber shank plate, the last to feature interchangeable IPS pillars, and an articulated chassis. The shoe was released on January 25, 2008, and was the last Air Jordan until the XX8 to have Roman numeral identification. The shoe was re-released in 2015-2016.                                                                                                                               |
| 15 | Air Jordan 2009    |   2009 | The Air Jordan 2009 was designed by Jason Mayden and was the first Air Jordan model named after the year of its release rather than its numbered system. Inspired by Jordan's defensive focus, the shoe incorporates Articulated Propulsion Technology used by Paralympian runners. It also features a durable pleated silk upper, protective Thermoplastic polyurethane (TPU) chassis, carbon fiber arch plate and Zoom Air structure. The shoe was released on January 31, 2009, and has not been re-released.                                                                                                                                                         |
| 16 | Air Jordan 2010    |   2010 | This was released during the 25th anniversary of the Air Jordan brand. The base of each midsole has stylized text that when combined reads: "I've failed over and over and over again in my life. And that is why I succeed." This quote is a reference to a 1997 advertising campaign, with Michael Jordan detailing his failures that led to his career successes.                                                                                                                                                                                                                                                                                                     |
| 17 | Air Jordan 2012    |   2012 | The Air Jordan 2012 offers six customization configurations. Two interchangeable sleeves and three insoles adapt to different playing styles. The Deluxe model was launched on February 8, while the customization Flight models were released on February 25, 2012. It was the final Air Jordan model to be named after the year it was released as the numbered system returned in 2013 with the Air Jordan XX8.                                                                                                                                                                                                                                                       |
| 18 | Air Jordan XX8     |   2013 | The Air Jordan XX8, designed by Tinker Hatfield, was released on February 16, 2013. The outside shroud was made from a Swiss fabric used for motorcycle jackets. [25]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
| 19 | Air Jordan XX9     |   2014 | The Air Jordan XX9, also designed by Hatfield, released in September 2014 in both an elephant print and a knit edition. The shoe debuted in the NBA by Russell Westbrook and Kawhi Leonard. The shoe has a performance woven upper, with areas that are stiff and others that are more flexible. [25] Jordan released two versions, a regular cut and low version. Some of the colorways released in low version are buckets, Chicago bulls, UNC, and infrared.                                                                                                                                                                                                          |
| 20 | Air Jordan XXX     |   2016 | The Air Jordan XXX was again designed by Tinker Hatfield. The first colorway of the shoe released on February 16. [45] The shoe consists of an upper and outsole similar to the XX9. The upper has a flyknit-constructed ankle collar that overextends slightly. The outsole has a more noticeable change, with a different traction pattern, while the midsole remains almost identical.                                                                                                                                                                                                                                                                                |
| 21 | Air Jordan XXXII   |   2017 | The Air Jordan XXXII was influenced by the Air Jordan 2 and included a Jordan "Wings" logo. It first released in the "Rossa Corsa" colorway on September 23, 2017. A "Banned" colorway was released on October 18, 2017. Another special edition called the "Russ" colorway was released to celebrate Russell Westbrook's sponsorship with Jordan Brand. Jordan Brand released 2 types of this shoe, the original mid length cut and low cut. [25]                                                                                                                                                                                                                       |
| 22 | Air Jordan XXXIII  |   2018 | The Air Jordan XXXIII was released on October 18, 2018. This is the first Air Jordan model to go laceless. [48]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
| 23 | Air Jordan XXXIV   |   2019 | The Air Jordan XXXIV was released on September 25, 2019, with details such as a commemorative date for the brand's 30th anniversary. [49]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                |
| 24 | Air Jordan XXXV    |   2020 | The Air Jordan XXXV debuted in the Fall of 2020 with a distinct new shape. The model focused on weight reduction for responsiveness. [50]  An area of the midfoot had a hole to provide stability, bounce, and comfort. [51]  The Air Jordan had several collaborations with contemporary NBA players, including Jayson Tatum and Zion Williamson.                                                                                                                                                                                                                                                                                                                       |
| 25 |                    |   2021 | The Air Jordan XXXVI was first teased by German-American professional basketball player Satou Sabally via social media, and made its on-court debut on March 25, 2021. [52]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
| 26 | Air Jordan XXXVII  |   2022 | The Air Jordan XXXVII was released on July 28, 2022. [53]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                |
| 27 | Air Jordan XXXVIII |   2023 | The Air Jordan XXXVIII contains a Cushlon 3.0 midsole along with a Zoom Strobel Unit. It was released on August                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
| 28 | Air Jordan XXXIX   |   2024 | The Air Jordan XXXIX was released in July 2024.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |

Columns: ['Air Jordan IV', '1989', 'In December 1988, Nike released the Air Jordan IV to the public. Designed by Tinker Hatfield, it was the first Air Jordan released on the global market. It had four colorways: White/Black, Black/Cement Grey, White/Fire Red-Black, and Off White/Military Blue. Nike featured director and actor Spike Lee in ads for the shoe. [27]  Lee had featured the shoe in his movie  Do The Right Thing . [25] Michael Jordan wore the Air Jordan IV when he made "The Shot", a series winner in Game 5 of the 1989 NBA First Round between the Chicago Bulls and the Cleveland Cavaliers. In 2012 a Cavalier colorway dubbed the "Cavs" was released to honor "The Shot".']
Shape: (29, 3)

--- Concatenated DataFrame Group 2 ---


|    | V   ·  T   ·  E   | Nike, Inc.                | [show]   |
|---:|:------------------|:--------------------------|:---------|
|  0 |                   | Links to related articles | [show]   |

Columns: ['V   ·  T   ·  E', 'Nike, Inc.', '[show]']
Shape: (1, 3)

--- Concatenated DataFrame Group 3 ---


...

Columns: ['Years', 'Sponsor', 'Bonuses']
Shape: (2, 3)

--- Concatenated DataFrame Group 5 ---


|    | †   | Denotes winners of the Triple Crown                                                |
|---:|:----|:-----------------------------------------------------------------------------------|
|  0 | *   | Denotes winners of the Derby and Preakness but not the Belmont                     |
|  1 | #   | Denotes winners of the other two combinations of 2 out of the 3 Triple Crown races |

Columns: ['†', 'Denotes winners of the Triple Crown']
Shape: (2, 2)

--- Concatenated DataFrame Group 6 ---


|    |   Year | Kentucky Derby   | Preakness Stakes   | Belmont Stakes   |
|---:|-------:|:-----------------|:-------------------|:-----------------|
|  0 |   1867 |                  |                    | Ruthless [Fy]    |

Columns: ['Year', 'Kentucky Derby', 'Preakness Stakes', 'Belmont Stakes']
Shape: (1, 4)
Processed 78f480a26ad6a8c7c064e9d9fd5b2c4f


In [ ]:
for source in processed_tables:
    print(f"{source} has {len(processed_tables[source])} tables")